In [1]:
# AOC - Day14 - Part 1 - final

In [2]:
import math

In [3]:
class Chemical:
    # need to do something special when this chemical is made
    # this is the ore, which gets mined, not made
    global o_str
    global ch
    def __init__(self, name, qty, rqty, reactants):
        self.name = name
        self.qty = qty
        self.rqty = rqty
        self.reactants = reactants
        return
    def make(self, req_qty):
        global ore_qty
        # if ore is being requested
        print("{}.make({})...".format(self.name, req_qty))
        if(self.name == o_str):
            # add this amount to accumulator
            ore_qty = ore_qty + req_qty
            # and return what is requested (instantly mined!)
            print("{} ORE mined! total so far: {}".format(req_qty, ore_qty))
            return(req_qty)
        # so ore not being requested.
        # see what is in inventory, and only kick off the reaction for what inventory can't provide
        need_amt = req_qty - self.qty
        if(need_amt <= 0):
            # have enough on hand.  reduce the inventory and return the requested amount.
            print("...enough on hand: {}".format(self.qty))
            self.qty -= req_qty
            return(req_qty)
        else:
            # determine factor required given need_amt and unit_quantity in the make reaction
            factor = calc_factor(need_amt, self.rqty)
            # now 'do the reaction' - iterate through the reactants
            for r in self.reactants:
                # get the ch array element for this reactant
                c = find_chem_by_name(ch, symnum(r)[0])
                req_amt = symnum(r)[1] * factor
                print("{} reactant: {} unit: {}, factor: {}, req_amt: {}, qty on-hand before make: {}".format(self.name, symnum(r)[0], symnum(r)[1], factor, req_amt, c.qty))
                made_amt = c.make(req_amt)
                # update on-hand leftover (req_amt is consumed in the calling reaction)
                c.qty = c.qty + made_amt - req_amt
                print("{} reactant: {} made_amt: {}, qty on-hand after make: {}".format(self.name, symnum(r)[0], made_amt, c.qty))
        return(factor*self.rqty)

In [4]:
class Reactant:
    def _init__(self, chem, qty):
        self.chem = chem
        self.qty = qty
        return

class Reaction:
    def __init__(self, reaction_tup):
        self.reactant_list = reaction_tup[0]
        self.product = reaction_tup[1]
        return

In [5]:
def parse_reaction(r_txt):
    # r_txt comes in like: "44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL"
    # parse this out and return a reactant list (r_list) and product (p)
    # each looks like  {'CHEM_SYMBOL': NUMBER}
    r_list = []
    p = {}
    left, right = r_txt.split(" => ")
    p_num, p_chem = right.split()
    p[p_chem] = int(p_num)
    for l in left.split(", "):
        reactant = {}
        reactant[l.split()[1]] = int(l.split()[0])
        r_list.append(reactant)
    return(r_list, p)

In [6]:
def calc_factor(req, makeunit):
    return(int(math.ceil(req / makeunit)))

In [7]:
# this function returns the first key, value of a dictionary as a tuple
def symnum(r):
    sym = list(r.keys())[0]
    num = list(r.values())[0]
    return((sym, num))

In [8]:
def find_chem_by_name(ch, name):
    c = [chem for chem in ch if chem.name == name]
    return(c[0])

In [9]:
# array of chemicals
ch = []
ore_qty = 0
# these are the strings used for the chemical symbols for Fuel and Ore
f_str = "FUEL"
o_str = "ORE"

In [10]:
# read the reactions list from the input file
ifn = "day14-input.txt"
# parse each line
# each line defines a chemical and contains the reactancts
with open(ifn) as f:
    for line in f:
        r_list, p = parse_reaction(line)
        # create a chemical for the product and add it to the chemical array
        # initial quantity is 0, of course
        ch.append(Chemical(symnum(p)[0], 0, symnum(p)[1], r_list))
# add ore as a reaction (ore => ore)
o_reaction_line = "1 {} => 1 {}".format(o_str, o_str)
r_list, p = parse_reaction(o_reaction_line)
ch.append(Chemical(symnum(p)[0], 0, symnum(p)[1], r_list))

In [11]:
# this will return the chemical element that is the fuel
x = find_chem_by_name(ch, f_str)

In [12]:
# this will invoke the making of 1 unit of fuel
a = x.make(1)

FUEL.make(1)...
FUEL reactant: CMKW unit: 1, factor: 1, req_amt: 1, qty on-hand before make: 0
CMKW.make(1)...
CMKW reactant: BRNWZ unit: 10, factor: 1, req_amt: 10, qty on-hand before make: 0
BRNWZ.make(10)...
BRNWZ reactant: JDBS unit: 4, factor: 2, req_amt: 8, qty on-hand before make: 0
JDBS.make(8)...
JDBS reactant: VKHN unit: 2, factor: 2, req_amt: 4, qty on-hand before make: 0
VKHN.make(4)...
VKHN reactant: ZWHT unit: 7, factor: 1, req_amt: 7, qty on-hand before make: 0
ZWHT.make(7)...
ZWHT reactant: JNDQ unit: 10, factor: 3, req_amt: 30, qty on-hand before make: 0
JNDQ.make(30)...
JNDQ reactant: BFKR unit: 4, factor: 5, req_amt: 20, qty on-hand before make: 0
BFKR.make(20)...
BFKR reactant: ORE unit: 115, factor: 5, req_amt: 575, qty on-hand before make: 0
ORE.make(575)...
575 ORE mined! total so far: 575
BFKR reactant: ORE made_amt: 575, qty on-hand after make: 0
JNDQ reactant: BFKR made_amt: 20, qty on-hand after make: 0
ZWHT reactant: JNDQ made_amt: 35, qty on-hand after make

WHRD reactant: ORE unit: 147, factor: 17, req_amt: 2499, qty on-hand before make: 0
ORE.make(2499)...
2499 ORE mined! total so far: 58663
WHRD reactant: ORE made_amt: 2499, qty on-hand after make: 0
MHQGW reactant: WHRD made_amt: 34, qty on-hand after make: 1
MHQGW reactant: DMSLX unit: 14, factor: 33, req_amt: 462, qty on-hand before make: 2
DMSLX.make(462)...
DMSLX reactant: ORE unit: 117, factor: 77, req_amt: 9009, qty on-hand before make: 0
ORE.make(9009)...
9009 ORE mined! total so far: 67672
DMSLX reactant: ORE made_amt: 9009, qty on-hand after make: 0
MHQGW reactant: DMSLX made_amt: 462, qty on-hand after make: 2
GMVJX reactant: MHQGW made_amt: 264, qty on-hand after make: 6
CZDM reactant: GMVJX made_amt: 130, qty on-hand after make: 2
CZDM reactant: PHNC unit: 1, factor: 65, req_amt: 65, qty on-hand before make: 0
PHNC.make(65)...
PHNC reactant: VGJG unit: 1, factor: 13, req_amt: 13, qty on-hand before make: 5
VGJG.make(13)...
VGJG reactant: BFKR unit: 1, factor: 1, req_amt: 1,

MJCD reactant: TJQDC made_amt: 20, qty on-hand after make: 4
MTJSD reactant: MJCD made_amt: 8, qty on-hand after make: 0
MTJSD reactant: RVKX unit: 2, factor: 4, req_amt: 8, qty on-hand before make: 1
RVKX.make(8)...
RVKX reactant: SFZTF unit: 4, factor: 3, req_amt: 12, qty on-hand before make: 0
SFZTF.make(12)...
SFZTF reactant: GSLTP unit: 9, factor: 3, req_amt: 27, qty on-hand before make: 0
GSLTP.make(27)...
GSLTP reactant: DFZRS unit: 1, factor: 3, req_amt: 3, qty on-hand before make: 1
DFZRS.make(3)...
DFZRS reactant: PWGXR unit: 17, factor: 1, req_amt: 17, qty on-hand before make: 0
PWGXR.make(17)...
PWGXR reactant: BFKR unit: 1, factor: 9, req_amt: 9, qty on-hand before make: 2
BFKR.make(9)...
BFKR reactant: ORE unit: 115, factor: 2, req_amt: 230, qty on-hand before make: 0
ORE.make(230)...
230 ORE mined! total so far: 96319
BFKR reactant: ORE made_amt: 230, qty on-hand after make: 0
PWGXR reactant: BFKR made_amt: 8, qty on-hand after make: 1
DFZRS reactant: PWGXR made_amt: 18,

In [13]:
print("the amount of ore to create 1 fuel is: {}".format(ore_qty))

the amount of ore to create 1 fuel is: 114125
